In [2]:
import  os 
import pandas as pd
from datetime import datetime,timedelta
import plotly.graph_objects as go
from google.cloud import bigquery

In [65]:
issues=pd.read_csv('https://exploitidf.github.io/IssuesTunnels/_downloads/caee2520ef1721f8437118adc2284ddf/issuesFermetures.csv')#'CodeEx', 'PC', 'Tatouage', 'triCode', 'Fermeture'
pcFr=issues[[ 'PC', 'Fermeture']].drop_duplicates()
pcFr['Fermeture']=pcFr['Fermeture']+'$'
pcFr=pcFr.groupby('PC')['Fermeture'].sum().str[:-1]
pcFr=pcFr.str.split('$')
PCs=pcFr.index
pcFr=list(pcFr)
pcFrDict={PCs[i]:pcFr[i] for i in range(4) }

frTa=issues[[ 'Fermeture','Tatouage']].drop_duplicates()
frTa['Tatouage']=frTa['Tatouage']+'$'
frTa=frTa.groupby('Fermeture')['Tatouage'].sum().str[:-1]
frTa=frTa.str.split('$')
FERMs=frTa.index
frTa=list(frTa)
frTaDict={FERMs[i]:frTa[i] for i in range(len(frTa)) }

In [26]:
issues[issues['Tatouage']=='L10.372X']

,CodeEx,PC,Tatouage,triCode,Fermeture
5,IS106,PCO,L10.372X,A14,A14&NEU-Y


In [51]:
      
client = bigquery.Client()
table =client.dataset("rapports_visites").table("LogDepot")
query = "SELECT * from `tunnels-dirif.rapports_visites.LogDepot` "
rows=client.query(query).result()
rowsTab=[list(row) for row in rows]
df=pd.DataFrame(rowsTab).drop_duplicates()
df.columns=['Tatouage', 'HoroDate', 'Agent', 'PC0', 'CM0', 'PC1', 'CM1', 'PC2', 'CM2', 'PC3', 'CM3', 'PC4', 'CM4', 'PC5', 'CM5', 'PC6', 'CM6', 
                  'PC7', 'CM7', 'PC8', 'CM8', 'PC9', 'CM9', 'PC10', 'CM10', 'PC11', 'CM11', 'PC12', 'CM12', 'PC13', 'CM13', 'PC14', 'CM14']
df['Tatouage']=df['Tatouage'].str.strip()
df['dt']=  pd.to_datetime(df['HoroDate'],format='%Y-%m-%d %H:%M:%S')
df['date']= (df['dt']-pd.Timedelta(hours=12)).dt.strftime('%d-%m')
df=df.join(issues.set_index('Tatouage')['CodeEx'],on='Tatouage')


In [60]:
ferm= 'NAN-I' # 'A14&NEU-Y'
dff=df[df['Tatouage'].isin(frTaDict[ferm])].copy()
isDt=dff[['CodeEx','date']].drop_duplicates()
isDt['date']=isDt['date']+'$'
isDt=isDt.groupby('CodeEx')['date'].sum().str[:-1].str.replace('$','  -  ').reset_index()
isDt.to_dict(orient='records')

KeyError: 'NAN-I'

In [ ]:
df[df['Tatouage'].isin(frTaDict[ferm])]

In [64]:
issues
frTaDict

{'A14&NEU-W': ['L10.396C',
  'L10.397D',
  'L10.401H',
  'L10.402J',
  'L10.403K',
  'L10.404L',
  'L10.405M',
  'L10.406N',
  'L10.407P',
  'L10.408R',
  'L10.409S',
  'L10.411U',
  'L10.412V',
  'L10.413W',
  'L10.414X',
  'L10.415Y',
  'L10.416A',
  'L10.417C',
  'L10.418D',
  'L10.419E',
  'L10.421G',
  'L10.422H',
  'L10.423J',
  'L10.424K',
  'L10.426M',
  'L10.435X',
  'L10.436Y',
  'L10.669C',
  'L10.834X',
  'L10.835Y',
  'L10.836A',
  'L10.837C',
  'L11.513F',
  'L11.514G'],
 'A14&NEU-Y': ['L10.366R',
  'L10.367S',
  'L10.368T',
  'L10.369U',
  'L10.371W',
  'L10.372X',
  'L10.373Y',
  'L10.374A',
  'L10.375C',
  'L10.376D',
  'L10.377E',
  'L10.378F',
  'L10.379G',
  'L10.381J',
  'L10.382K',
  'L10.383L',
  'L10.384M',
  'L10.385N',
  'L10.386P',
  'L10.388S',
  'L10.395A',
  'L10.398E',
  'L10.399F',
  'L10.425L',
  'L10.668A'],
 'BAP&SCL-W': ['L10.625Y',
  'L10.626A',
  'L10.627C',
  'L10.628D',
  'L10.629E',
  'L10.631G',
  'L10.632H',
  'L10.633J',
  'L10.641T',
  'L10.

In [32]:
df[:2]

,Tatouage,HoroDate,Agent,PC0,CM0,PC1,CM1,PC2,CM2,PC3,...,PC10,CM10,PC11,CM11,PC12,CM12,PC13,CM13,PC14,CM14
0,L10.372X,2024-02-19 00:23:00,Kamel,R2,ComAl4_Qûàtré,R3,ComAl2_Trop Û,R1,ComAl2_Trop Û,R1,...,R0,ComAl4_Qûàtré,R2,ComAl2_Trop Û,R2,ComAl0_Très,R2,ComAl2_Trop Û,R1,ComAl0_Très
1,L10.371W,2024-02-19 02:32:00,Karl,R4,ComAl2_Trop Û,R9,ComAl3_TroisÈ^cî,R1,ComAl0_Très,R5,...,R1,ComAl4_Qûàtré,R0,ComAl0_Très,R2,ComAl3_TroisÈ^cî,R1,ComAl1_îàèù,R1,ComAl4_Qûàtré
